In [10]:
import plotly.graph_objs as go
from ipywidgets import interact
from plotly.offline import init_notebook_mode, iplot
from sklearn.datasets import make_regression
import numpy as np


In [11]:
init_notebook_mode(connected=True)


<script>requirejs.config({paths: { 'plotly': ['https://cdn.plot.ly/plotly-latest.min']},});if(!window.Plotly) {{require(['plotly'],function(plotly) {window.Plotly=plotly;});}}</script>

In [12]:
def compute_cost_function(theta, x, y):
    x_arr = np.asarray(x) if type(x) is list else x
    m = x_arr.shape[0]
    ones = np.ones((m, 1))
    xa = np.hstack((ones, x))
    approx_value = np.sum(theta * xa, axis=1)
    dif = approx_value - y
    return sum(dif ** 2) / (2 * m)


def compute_gradient(theta, x, y):
    x_arr = np.asarray(x) if type(x) is list else x
    m = x_arr.shape[0]
    ones = np.ones((m, 1))
    xa = np.hstack((ones, x))
    approx_value = np.sum(theta * xa, axis=1)
    dif = approx_value - y
    grad0 = np.asarray([sum(dif) / m])
    gradn = np.dot(np.asarray(dif), x) / m
    grad = np.concatenate((grad0, gradn), axis=0)
    return grad


def propose_theta(data):
    return np.ones(data.shape[1])


def update_theta(theta, gradient, alpha):
    return theta - alpha * gradient


def calc_theta_for_batch(batch_data, min_theta, alpha):
    batch_n = batch_data.shape[1] - 1
    batch_x = batch_data[:, range(batch_n)]
    batch_y = batch_data[:, batch_n]
    g = compute_gradient(min_theta, batch_x, batch_y)
    return update_theta(min_theta, g, alpha)


def mini_batch_gradient_descent(data, start_theta=None, alpha=None, max_iter=None, batch_size=5):
    if start_theta is None:
        start_theta = propose_theta(data)

    if alpha is None:
        alpha = 0.001

    if max_iter is None:
        max_iter = 100

    min_theta = start_theta
    min_cost = float("inf")
    f_n = data.shape[1] - 1

    iteration_without_improvement = 0
    cost_history = []
    cur_alpha = alpha
    while iteration_without_improvement < max_iter:
        np.random.shuffle(data)
        batch_data = data[0:batch_size]
        cur_theta = calc_theta_for_batch(batch_data, min_theta, cur_alpha)
        x = data[:, range(f_n)]
        y = data[:, f_n]
        cur_cost = compute_cost_function(cur_theta, x, y)
        cost_history.append(cur_cost)
        if cur_cost < min_cost:
            min_cost = cur_cost
            min_theta = cur_theta
            iteration_without_improvement = 0
            cur_alpha *= 0.95
        else:
            iteration_without_improvement+=1

    return min_theta, min(cost_history)



In [17]:

data = make_regression(200, n_features=2)


@interact(mi=(0,2000,50), regenerate_input_data=False)
def find_and_display_regression(mi, regenerate_input_data):
    input_data = data
    if regenerate_input_data:
        input_data = make_regression(200, n_features=2)
      
    x = np.transpose(input_data[0])[0]
    y = np.transpose(input_data[0])[1]
    z = input_data[1]
    r = np.column_stack((x,y,z))    
    theta, cost_minimum = mini_batch_gradient_descent(r, alpha=0.05, max_iter=mi, batch_size=140)
    print(theta)
    print(cost_minimum)
    t1 = range(-5,8)
    t2 = range(-5,8)
    xv, yv = np.meshgrid(t1, t2)
    xvf = xv.flatten()
    yvf = yv.flatten()
    theta0 = theta[0]
    theta1 = theta[1]
    theta2 = theta[2]
    f=theta0+theta1*xvf+theta2*yvf 
    
    data_trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        name='Input data'
    )
    
    reg_trace=go.Mesh3d(x=xvf,y=yvf,z=f,color='red',opacity=0.5)
    
    camera = dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=0.5, y=1, z=0.5)
    )
    
    fig = go.Figure(data=[data_trace, reg_trace])
    
    fig['layout'].update(
        scene=dict(camera=camera)
    )

    iplot(fig)


interactive(children=(IntSlider(value=1000, description='mi', max=2000, step=50), Checkbox(value=False, descri…

[ 2.62895451  7.1006604  41.18625235]
349.4533303918012


<div id="ec3bed78-383e-4953-88a1-8dd8797456c6" style="height: 525px; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">require(["plotly"], function(Plotly) { window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("ec3bed78-383e-4953-88a1-8dd8797456c6", [{"mode": "markers", "name": "Input data", "x": [1.0766062975582766, -1.5203493525115663, 0.9973434192914057, -0.3781981595318158, 1.387479234551105, 0.10946117347350699, -0.08692376225814492, 1.4489060015759327, 2.202629758137366, -1.3347803481942386, 0.8529001843882378, 1.071461593494217, -0.7703931093558558, 1.2724350715412602, -0.4233726272060751, 1.0421615842139298, 0.11037345737903315, 1.2119016576944246, -0.5678021678948376, -0.2551243870694487, 0.9551946617431334, -0.6100264936980405, -0.6533676864134813, -0.6067576905106654, -0.7541968532162656, -0.15645510802494625, 0.3953090597038179, 0.009419039369158763, 0.69952488774426, 0.8285003206679163, 0.37536093596279785, -0.7327239050375459, 0.27289528126741264, -1.653168632786268, 0.061348816098770964, -2.130543886410104, 0.4409068583399459, -1.7217713650623585, -0.3879017541332277, 1.1294807591981066, -0.9553915373352345, -0.05602630442096234, 1.062109818029398, -0.683931845553696, 1.2046686057302276, 1.2992384920840099, 0.5408825004286105, -0.03801057257649888, 0.7462491882143669, -0.7408915949424583, 0.6118565772607446, -1.0681032850138614, 0.46677916058138585, -1.498403051936495, -0.22576866620330063, -0.14540569944525134, 1.1667222420429226, 1.9299997074172663, -1.590580174735802, -0.214467412507122, -0.38796209733272435, -0.4371871001001775, -1.8543151863707832, -0.3810601813996093, 1.1257489237521023, -0.6201247233400838, -0.7650538047802417, 0.8125204644559261, 1.2039517223591392, 0.5455555243098658, 1.2442440382016502, -0.768279108217954, -0.289354338824301, 1.0092782786054157, 0.36337933554545154, 0.6605930249892159, 1.1259834676661669, 1.4613932145248842, 0.21567083001309292, 0.5804098303396981, 0.9771387395468786, -2.0875237226207566, 0.36658659693027873, -0.7182328473299994, -0.3153277268706897, 1.1162976855430746, 1.4063494168747512, -0.614760581353303, 2.7942216708441165, 0.1704462075284727, -1.128451991050711, -0.029734291675793754, -0.09627469321153392, -1.4884158004554273, -1.3449562884208046, -0.6157540849978654, -1.4909160135172748, 0.8188233219711464, -0.898336809393073, 1.0494527957177027, 0.6431929542815054, -0.8677729356734408, 1.4081122978548621, -0.11521294550654942, -1.3937742066837777, 0.11624370691046892, 0.7748880969627846, 1.0434479325913508, -1.4330182207932267, -0.4510726931240955, -1.7007653828665175, 0.75174062645551, 1.252624322033825, -1.009074171026464, 0.682339637261446, -1.3536731274039768, -0.6543358004914107, 0.27301640624602097, -0.14850220774700595, 1.4486312796823952, 0.6664424812952031, -0.666241390696679, 0.25804548619300344, 0.800437702418552, -0.5930804408329342, 0.9509978540298069, 1.4753056381565832, 0.594020172508931, 1.3548192011218816, 0.25533649926697566, 0.059925083062141525, -1.1280262348281584, -0.11903610412493629, -0.6989110002139743, -0.18323401324576352, 1.2935407439706228, 0.6122655806349445, -0.8483421315419232, -0.055736913101136924, -1.237040214808129, 0.4774656043827873, -0.6076550472464182, -0.04721262251378314, 0.6298708918303253, 0.07373389003424101, -2.1496411970340916, 0.5692042808027798, -1.2511367935610853, 0.9900888026856924, 1.1441628033192213, 0.8633610896666681, -0.7041322702052504, -0.06536398605164935, 1.9251862404568165, 0.044155436573997356, 0.9969953764006211, -1.9459268274236308, 1.736126052443753, -0.043166127924799215, -0.7015377017803448, -0.5775110278817517, -0.6292988746943764, -1.189492372537645, 0.3680329947325886, -1.137172114871948, 1.2025228996165063, -1.2370442989037647, -0.6961718350530863, 0.42750709651823315, 2.299207608818678, -0.24783063932602753, -0.4552612061437024, 0.6956398363503458, -0.0070395241338285066, -0.6049906298229327, 1.2322395512794415, -0.63

<div id="d396b7f5-57ac-42a2-8051-93f9342fde2c" style="height: 525px; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">require(["plotly"], function(Plotly) { window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("d396b7f5-57ac-42a2-8051-93f9342fde2c", [{"mode": "markers", "name": "Input data", "x": [0.26105797301269823, 0.40916373821594093, -1.7381662879151096, 0.7981556392023964, 0.8431771859358402, -1.1708817902153248, -2.6561450416109555, 1.4295806270303415, -1.2595394546382053, 0.3747137586886551, 2.2437448841084637, 0.17211497410769427, -0.7612692414568294, -0.018229619874537695, -0.34053774004438525, 0.09388194603788333, -0.9247012377397339, 0.6412351104380892, -0.03729588242766651, -0.4207890960225668, 1.7379135675761632, 0.5733209471915582, 0.8741790821071981, 1.6097502476046937, 1.452873486343366, -0.17633588665826722, -0.021636239739273064, 0.734867007630567, 1.0310260077848818, -0.10802391402164148, 0.11711329723202867, 0.9532489867738875, 0.28053457068649457, 0.5919022323734409, 0.5936120576201667, 2.517156658743168, 1.4681034099467176, 1.7968201588564954, 0.1211055984125724, 0.5083325805239489, 2.5196400068456533, 0.018794939580386825, -0.9804916024504591, 0.2706316792344275, -1.2246300247606345, -0.32725027481844127, 0.9696095266526512, 0.01603103554358082, 0.3004727055853047, -1.82929382438673, 0.28036602648807546, 1.42682241910217, 0.1104154227549872, 0.46610685182923894, 1.968803622495039, -0.634903986276872, -1.4998378434562454, -0.4491554154177956, 0.27890922257481543, -0.7995334257133931, -0.2145743215536346, 0.3869267340819143, -0.4345510687221896, -2.1323650967289054, -0.05835085989554676, -0.11620403314092373, -0.8727777037819588, 0.33344389669231933, 1.9456738537169422, 0.8289437655350154, -1.8464316668240883, -0.3282868325725871, 0.37536638879264234, 0.012654171083352577, 1.1711677162139398, -1.2887660877335272, 1.1092146035413513, -1.6766066194145386, 0.07432748052161156, 0.006792660291631456, -0.9597937624618054, 0.866913175961926, -0.633560576017824, -0.951412271180529, -0.40262627205830365, 1.7553128879877884, -1.7321145466344685, -2.11115504130188, -0.2638731561182573, -0.6335643832687834, 0.44781932946141234, 0.5635682742931789, -0.6151347618798249, 0.10215930852865238, -0.13587082879946127, 0.0907046158790495, -0.7368468129622675, 1.3565760823638668, -0.014220135094626014, 1.4700108536941443, -2.968343496975412, -1.2440950168968081, -0.8758770202280378, -0.6373977816618506, -0.9981749207879584, -1.3218924127757494, -1.4364076403222181, 0.20881434326557055, -0.9326981572386617, 0.5593782002619601, 0.5831169600820947, -0.16839201473702164, 1.2270009729699058, -1.2124853780728475, 0.2942123228238119, 1.224055386498124, -0.14131405570323222, 0.4191744756918495, 0.17351665880912234, -2.6666686940295716, 1.5435279602964382, -0.894044223243252, -1.0267763421919518, 2.5712466801948173, 0.11588654380621777, 0.45130456354604126, 1.5808752429605193, 1.2283843956421616, -0.28861076104723465, -0.23392821107704906, 1.8391002875376397, -0.5816339202783487, 0.4256659040224516, 0.7197394204580643, 0.6499147265719932, 0.6761308554247992, 1.1994785505191246, -0.4296348869976804, 0.14751511050024962, -1.8191498809702553, 1.4795069153336204, -0.35741124671479535, -0.7995531090086953, 0.13407895634655964, -1.1099040769971866, -0.08681319287353838, 1.2987714165571738, -0.8332748171013159, -0.9498753441392975, -0.6637858679764028, -0.9356187133977318, -0.6501355413455059, -1.0920523151186532, 1.0258569571961094, -0.9609213533433535, -0.6431994110832422, -1.6652079112519582, 0.7176682105429183, -1.8818113356167214, 0.2458875420766178, -1.3952749377612526, -0.44955018894249776, -0.3163551131280183, 1.3496045053803174, 0.5163453517913424, 0.8611091843512838, -0.08997311268233776, -1.6874589426868534, 1.4963644734196528, -1.0996026824977367, 0.38218765102116437, 0.2210258212963767, 0.5990003016057833, 0.5160567692149786, 0.32828995706800856, -1.14637685